<a href="https://colab.research.google.com/github/kridtapon/Kumo-Breakout-Momentum/blob/main/Kumo_Breakout_Momentum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install vectorbt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.6/527.6 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 28.0 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt

# Function to calculate Ichimoku Cloud components
def ichimoku_cloud(df):
    """
    Calculate Ichimoku Cloud components.
    """
    # Tenkan-sen (Conversion Line)
    period_tenkan = 9
    tenkan_sen = (df['High'].rolling(window=period_tenkan).max() + df['Low'].rolling(window=period_tenkan).min()) / 2

    # Kijun-sen (Base Line)
    period_kijun = 26
    kijun_sen = (df['High'].rolling(window=period_kijun).max() + df['Low'].rolling(window=period_kijun).min()) / 2

    # Senkou Span A (Leading Span A)
    senkou_span_a = ((tenkan_sen + kijun_sen) / 2).shift(period_kijun)

    # Senkou Span B (Leading Span B)
    period_senkou_b = 52
    senkou_span_b = ((df['High'].rolling(window=period_senkou_b).max() + df['Low'].rolling(window=period_senkou_b).min()) / 2).shift(period_kijun)

    # Chikou Span (Lagging Span)
    chikou_span = df['Close'].shift(-period_kijun)

    return tenkan_sen, kijun_sen, senkou_span_a, senkou_span_b, chikou_span

# Define stock symbol and time period
symbol = 'META'
start_date = '2019-01-01'
end_date = '2025-01-01'

# Download the data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']

# Calculate Ichimoku Cloud components
df['Tenkan'], df['Kijun'], df['Senkou_A'], df['Senkou_B'], df['Chikou'] = ichimoku_cloud(df)

# Identify Kumo Twist (Bearish to Bullish & Bullish to Bearish)
df['Bullish_Kumo_Twist'] = (df['Senkou_A'] > df['Senkou_B']) & (df['Senkou_A'].shift(1) < df['Senkou_B'].shift(1))
df['Bearish_Kumo_Twist'] = (df['Senkou_A'] < df['Senkou_B']) & (df['Senkou_A'].shift(1) > df['Senkou_B'].shift(1))

# Entry and Exit signals based on Kumo Twist and Chikou Span
df['Entry'] = df['Bullish_Kumo_Twist'] & (df['Chikou'] > df['Close'])
df['Exit'] = df['Bearish_Kumo_Twist'] & (df['Chikou'] < df['Close'])

# Filter data for test period (2020-2025)
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]

# Backtest using vectorbt
portfolio = vbt.Portfolio.from_signals(
    close=df['Close'],
    entries=df['Entry'],
    exits=df['Exit'],
    init_cash=100_000,
    fees=0.001
)

# Display performance metrics
print(portfolio.stats())

# Plot equity curve
portfolio.plot().show()

[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning: Metric 'sharpe_ratio' requires frequency to be set
  warnings.warn(warning_message)
/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning: Metric 'calmar_ratio' requires frequency to be set
  warnings.warn(warning_message)
/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning: Metric 'omega_ratio' requires frequency to be set
  warnings.warn(warning_message)
/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning: Metric 'sortino_ratio' requires frequency to be set
  warnings.warn(warning_message)


Start                         2020-01-02 00:00:00
End                           2024-12-31 00:00:00
Period                                       1258
Start Value                              100000.0
End Value                            640356.76901
Total Return [%]                       540.356769
Benchmark Return [%]                   180.172856
Max Gross Exposure [%]                      100.0
Total Fees Paid                        938.964445
Max Drawdown [%]                        18.426435
Max Drawdown Duration                       353.0
Total Trades                                    4
Total Closed Trades                             3
Total Open Trades                               1
Open Trade PnL                      486036.713451
Win Rate [%]                                100.0
Best Trade [%]                          24.306387
Worst Trade [%]                          9.511028
Avg Winning Trade [%]                    15.74191
Avg Losing Trade [%]                          NaN


In [3]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt

# Function to calculate Kumo components
def calculate_kumo(df, period_tenkan=9, period_kijun=26, period_senkou_b=52):
    # Tenkan-sen (Conversion Line)
    tenkan_sen = (df['High'].rolling(window=period_tenkan).max() + df['Low'].rolling(window=period_tenkan).min()) / 2

    # Kijun-sen (Base Line)
    kijun_sen = (df['High'].rolling(window=period_kijun).max() + df['Low'].rolling(window=period_kijun).min()) / 2

    # Senkou Span A (Leading Span A)
    senkou_span_a = ((tenkan_sen + kijun_sen) / 2).shift(period_kijun)

    # Senkou Span B (Leading Span B)
    senkou_span_b = ((df['High'].rolling(window=period_senkou_b).max() + df['Low'].rolling(window=period_senkou_b).min()) / 2).shift(period_kijun)

    # Chikou Span (Lagging Span)
    chikou_span = df['Close'].shift(-period_kijun)

    return tenkan_sen, kijun_sen, senkou_span_a, senkou_span_b, chikou_span

# Define the stock symbol and time period
symbol = 'META'  # Replace with any other stock symbol
start_date = '2019-01-01'
end_date = '2025-01-01'

# Download the stock data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']

# Calculate Kumo components
df['Tenkan'], df['Kijun'], df['Senkou_A'], df['Senkou_B'], df['Chikou'] = calculate_kumo(df)

# Define Entry and Exit signals based on Kumo twist
df['Entry'] = (df['Senkou_A'].shift(1) < df['Senkou_B'].shift(1)) & (df['Chikou'] > df['Close'])
df['Exit'] = (df['Senkou_A'].shift(1) > df['Senkou_B'].shift(1)) & (df['Chikou'] < df['Close'])

# Filter data for the test period (2020-2025)
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]

# Backtest using vectorbt
portfolio = vbt.Portfolio.from_signals(
    close=df['Close'],
    entries=df['Entry'],
    exits=df['Exit'],
    init_cash=100_000,
    fees=0.001
)

# Display performance metrics
print(portfolio.stats())

# Plot equity curve
portfolio.plot().show()


[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



Start                         2020-01-02 00:00:00
End                           2024-12-31 00:00:00
Period                                       1258
Start Value                              100000.0
End Value                           474595.108001
Total Return [%]                       374.595108
Benchmark Return [%]                   180.172856
Max Gross Exposure [%]                      100.0
Total Fees Paid                       4163.170808
Max Drawdown [%]                        49.193602
Max Drawdown Duration                       381.0
Total Trades                                   10
Total Closed Trades                            10
Total Open Trades                               0
Open Trade PnL                                0.0
Win Rate [%]                                 80.0
Best Trade [%]                           92.44771
Worst Trade [%]                        -33.562407
Avg Winning Trade [%]                   33.645285
Avg Losing Trade [%]                   -23.368712
